Setup globals

In [1]:
import datetime
import pandas as pd
import sqlite3
import time

day_of_seconds = 86400
week_of_seconds = 7 * day_of_seconds
month_of_seconds = 30 * day_of_seconds

today = int(time.mktime(datetime.date.today().timetuple()))

The following cell values are safe to modify as needed

In [2]:
sqlite_file = 'RaMBLE.sqlite'
max_rows = 500

# Typically only looking back the past week
wayback = today-week_of_seconds

Pull in database

In [3]:
pd.set_option('display.max_rows', max_rows)
db = sqlite3.connect(sqlite_file)
new_devices = pd.read_sql_query("SELECT * from devices WHERE first_seen >= %d" % (wayback), db)
old_devices = pd.read_sql_query("SELECT * from devices WHERE first_seen < %d" % (wayback), db)

The database has two important tables: `devices` and `locations`, but this notebook will only be focusing on `devices`.

devices
* id (int)
* address (string: OUI)
* adv_flags (float)
* device_name (string)
* device_type (string)
* first_seen (int)
* last_seen (int)
* msd_key (float, Manufacturer Data: ID)
* msd_field (hex string, Manufacturer Data: Data)
* service_uuids (string uuid)
* service_data (string hex data)
* days_seen (int)
* raw_adv_data (bitstring)

locations
* id (int)
* device_id (int)
* timestamp (int)
* rssi (int)
* tx_power (float)
* latitude (float)
* logitude (float)
* accuracy (float)
* packets_received (int)

# New Devices

In [4]:
print(f"{old_devices.id.count()} devices from {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(old_devices.iloc[0].first_seen))} to {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(old_devices.iloc[-1].first_seen))}")
print(f"{new_devices.id.count()} devices from {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(new_devices.iloc[0].first_seen))} to {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(new_devices.iloc[-1].first_seen))}")

10545 devices from 2018-11-20 15:12:37 to 2020-03-13 17:39:36
1156 devices from 2020-08-31 18:06:10 to 2020-09-05 13:31:39


## New Device Names

In [5]:
old_names = old_devices['device_name'].dropna(how='any').unique()

new_names = new_devices.copy()
for old_name in old_names:
    new_names = new_names[new_names.device_name != old_name]

name_count = pd.DataFrame(new_names.device_name.value_counts()).sort_index()

device_report = list()

for new_name in name_count.index:
    new_device = new_devices[new_devices.device_name == new_name]
    
    for device in new_device.index:
        device_report.append(new_device.loc[device, ["device_name", "device_type", "address", "service_uuids"]].values)

device_name_report = pd.DataFrame(device_report)
device_name_report.columns = ["device_name", "device_type", "address", "service_uuids"]

In [6]:
name_count

,device_name
00A05020BEF01B28z22,1
00A05020E1A50D200T0,1
00A05020E1B20028z02,1
ALAM (24:F7:2C) ,1
BLE-OHL-01,1
BLE_08A0,1
BLE_891B,1
BLE_92EB,1
BLE_C826,1
BLE_E586,1


In [7]:
device_name_report.sort_values(by="address")

,device_name,device_type,address,service_uuids
0,00A05020BEF01B28z22,Battery Service,00:01:AD:66:08:66,180f
2,00A05020E1B20028z02,Battery Service,00:01:AD:71:87:09,180f
24,G-519C9C,None,00:05:C2:51:9C:9C,47726f74-6547-4c4f-5353-414253000000
39,Micro Mechanic,None,00:1D:A5:00:F9:57,fff0
45,PayPal-10395335,None,00:1D:FA:20:09:31,None
36,MSH317_Ble_C7A,CUBE TECHNOLOGIES,00:22:6C:D6:8C:7B,"ffc0,6666"
1,00A05020E1A50D200T0,Battery Service,00:A0:50:20:E1:A5,180f
47,RN456D,None,04:91:62:29:45:6D,1a6b093f-3e81-4e2a-bad8-67dbd81acfbe
31,LG CJ45 (70),LG Electronics,08:EF:3B:54:92:70,None
33,LG CJ45 (F1),LG Electronics,08:EF:3B:7B:18:F1,None


## New Device Types

In [8]:
old_types = old_devices['device_type'].dropna(how='any').sort_values().unique()
new_types = new_devices.copy()

for old_type in list(old_types):
    new_types = new_types[new_types.device_type != old_type]

type_count = pd.DataFrame(new_types.device_type.value_counts()).sort_index()

device_report = list()

for new_type in type_count.index:
    new_device = new_devices[new_devices.device_type == new_type]
    
    for device in new_device.index:
        device_report.append(new_device.loc[device, ["device_name", "device_type", "address", "service_uuids"]].values)

device_type_report = pd.DataFrame(device_report)
device_type_report.columns = ["device_name", "device_type", "address", "service_uuids"]

In [9]:
type_count

,device_type
"Apple, Wireless Audio",4
Bose,3
"Bose, SGL Italia",2
CSR,1
CUBE TECHNOLOGIES,3
Device Information,6
"Device Information, Fitbit",1
Eddystone UID beacon,1
Eddystone URL beacon - http://mrefer.com/4153232615,1
Exposure Notification,1


In [10]:
device_type_report.sort_values(by="address")

,device_name,device_type,address,service_uuids
12,MSH317_Ble_C7A,CUBE TECHNOLOGIES,00:22:6C:D6:8C:7B,"ffc0,6666"
58,None,"Hewlett-Packard Company, HP",00:68:EB:FB:22:E3,fe78
60,None,"Hewlett-Packard Company, HP",02:68:EB:45:2C:20,fe78
57,None,"Hewlett-Packard Company, HP",02:68:EB:46:80:9B,fe78
8,LE-Bose Color II SoundLink,"Bose, SGL Italia",28:11:A5:AB:12:E5,febe
7,LE-Bose Color II SoundLink,"Bose, SGL Italia",2C:41:A1:CC:30:E6,febe
22,None,Exposure Notification,35:BE:AB:BF:94:69,fd6f
62,None,"Hewlett-Packard Company, HP",3A:22:E2:9E:58:99,fe78
25,None,"Google, Chromecast",40:E2:B3:BD:6B:08,fe9f
47,None,"Google, Chromecast",42:7F:61:FD:D2:80,fe9f


## New OUIs

In [11]:
old_ouis = list()
new_ouis = list()
new_addresses = list()

for old_address in old_devices['address']:
    oui_octets = old_address.split(':')[:3]
    
    # check to see if this is a global oui
    if int(oui_octets[0], 16) & 0x2 == 0:
        #turn off the multicast bit
        oui_octets[0] = '{:02x}'.format(int(oui_octets[0], 16) & 0xe)
        old_ouis.append(':'.join(oui_octets))

for new_address in new_devices['address']:
    oui_octets = new_address.split(':')[:3]
    
    # check to see if this is a global oui
    if int(oui_octets[0], 16) & 0x3 > 0:
        continue
    
    #turn off the multicast bit
    oui_octets[0] = '{:02x}'.format(int(oui_octets[0], 16) & 0xe)
    new_oui = ':'.join(oui_octets)
    
    if new_oui not in old_ouis:
        new_ouis.append(new_oui)
        new_addresses.append(new_address)

device_report = list()

for new_address in new_addresses:
    new_device = new_devices[new_devices.address == new_address]
    
    for device in new_device.index:
        device_report.append(new_device.loc[device, ["device_name", "device_type", "address", "service_uuids"]].values)

device_oui_report = pd.DataFrame(device_report)
device_oui_report.columns = ["device_name", "device_type", "address", "service_uuids"]

In [12]:
pd.DataFrame(pd.Series(new_ouis).value_counts())

,0
04:FE:FB,3
08:27:37,3
00:01:AD,2
04:5C:E9,2
08:6F:8E,1
08:BC:89,1
00:34:F5,1
04:77:5F,1
04:2A:80,1
08:E2:44,1


In [13]:
device_oui_report.sort_values(by="address")

,device_name,device_type,address,service_uuids
9,00A05020BEF01B28z22,Battery Service,00:01:AD:66:08:66,180f
112,00A05020E1B20028z02,Battery Service,00:01:AD:71:87:09,180f
222,G-519C9C,None,00:05:C2:51:9C:9C,47726f74-6547-4c4f-5353-414253000000
120,Micro Mechanic,None,00:1D:A5:00:F9:57,fff0
100,MSH317_Ble_C7A,CUBE TECHNOLOGIES,00:22:6C:D6:8C:7B,"ffc0,6666"
260,None,Microsoft,00:26:9D:67:3A:1F,None
140,None,"Hewlett-Packard Company, HP",00:68:EB:FB:22:E3,fe78
79,00A05020E1A50D200T0,Battery Service,00:A0:50:20:E1:A5,180f
155,None,Microsoft,08:02:A2:3A:43:01,None
196,None,Microsoft,08:B3:D8:50:91:21,None


## New Manufacturer ID's

In [14]:
old_msds = old_devices['msd_key'].dropna(how='any').sort_values().unique()
new_msds = new_devices.copy()

for old_msd in list(old_msds):
    new_msds = new_msds[new_msds.msd_key != old_msd]

msd_table = pd.DataFrame(new_msds.msd_key.value_counts())

# Convert msd_key into hex
msd_counts = list()

for msd_key in msd_table.index:
    msd_counts.append(['0x{:02x}'.format(int(float(msd_key))),msd_key, msd_table.loc[msd_key, "msd_key"]])

msd_counts = pd.DataFrame(msd_counts)
msd_counts.columns = ["Manufacturer ID (hex)", "Manufacturer ID (raw)", "Count"]

device_report = list()

for new_msd in msd_counts["Manufacturer ID (raw)"]:
    new_device = new_devices[new_devices.msd_key == new_msd]
    
    for device in new_device.index:
        device_report.append(new_device.loc[device, ["device_name", "device_type", "address", "service_uuids", "msd_key"]].values)

device_msd_report = pd.DataFrame(device_report)
device_msd_report.columns = ["device_name", "device_type", "address", "service_uuids", "Manufacturer ID (raw)"]

In [15]:
msd_counts

,Manufacturer ID (hex),Manufacturer ID (raw),Count
0,0x7544,30020.0,2
1,0x5c7,1479.0,1
2,0x3141,12609.0,1
3,0xa212,41490.0,1
4,0x1a5,421.0,1


In [16]:
device_msd_report.sort_values(by="address")

,device_name,device_type,address,service_uuids,Manufacturer ID (raw)
0,Dual iPlug,None,21:E1:63:A3:42:41,None,30020.0
1,Dual iPlug,None,41:42:D3:BB:C2:14,None,30020.0
4,None,None,88:D0:39:98:C7:E4,None,41490.0
3,DCS-8525LH-4270,None,B0:C5:54:50:42:70,3910bf96-4348-f3e4-a002-6afb7fc989ae,12609.0
5,I_TL,Icon Health and Fitness,DA:5D:5A:48:68:17,00001533-1412-efde-1523-785feabcd123,421.0
2,LCIRemoteNwBKfKvbV,Lippert Components,F3:FE:46:53:CA:8F,00000000-0200-a58e-e411-afe28044e62c,1479.0


## New Service UUIDs

In [17]:
old_uuids = old_devices['service_uuids'].dropna(how='any').unique()

new_uuids = new_devices.copy()
for old_uuid in old_uuids:
    new_uuids = new_uuids[new_uuids.service_uuids != old_uuid]

device_report = list()

for new_msd in msd_counts["Manufacturer ID (raw)"]:
    new_device = new_devices[new_devices.msd_key == new_msd]
    
    for device in new_device.index:
        device_report.append(new_device.loc[device, ["device_name", "device_type", "address", "service_uuids", "msd_key"]].values)

device_msd_report = pd.DataFrame(device_report)
device_msd_report.columns = ["device_name", "device_type", "address", "service_uuids", "Manufacturer ID (raw)"]

uuid_count = pd.DataFrame(new_uuids.service_uuids.value_counts()).sort_index()

device_report = list()

for new_uuid in uuid_count.index:
    new_device = new_devices[new_devices.service_uuids == new_uuid]
    
    for device in new_device.index:
        device_report.append(new_device.loc[device, ["device_name", "device_type", "address", "service_uuids"]].values)

device_uuid_report = pd.DataFrame(device_report)
device_uuid_report.columns = ["device_name", "device_type", "address", "service_uuids"]

In [18]:
uuid_count

,service_uuids
0000,1
00000000-0200-a58e-e411-afe28044e62c,1
00001533-1412-efde-1523-785feabcd123,1
3910bf96-4348-f3e4-a002-6afb7fc989ae,1
447c0000-ed49-fe29-71e1-87a4e15ecff6,1
47726f74-6547-4c4f-5353-414253000000,1
6e400001-b5a3-f393-e0a9-e50e24dcca9e,1
88e0,1
fd6f,1
"feaa,1786b941-062b-17b7-4df6-3f79361ad91f",1


In [19]:
device_uuid_report.sort_values(by="address")

,device_name,device_type,address,service_uuids
5,G-519C9C,None,00:05:C2:51:9C:9C,47726f74-6547-4c4f-5353-414253000000
11,MSH317_Ble_C7A,CUBE TECHNOLOGIES,00:22:6C:D6:8C:7B,"ffc0,6666"
7,Mercury,None,0A:F9:0A:41:05:00,88e0
8,None,Exposure Notification,35:BE:AB:BF:94:69,fd6f
3,DCS-8525LH-4270,None,B0:C5:54:50:42:70,3910bf96-4348-f3e4-a002-6afb7fc989ae
4,PDSCB110,None,D0:C5:D3:EF:55:BE,447c0000-ed49-fe29-71e1-87a4e15ecff6
6,iRadar,nRF UART Service,D0:C6:6E:BD:5D:39,6e400001-b5a3-f393-e0a9-e50e24dcca9e
2,I_TL,Icon Health and Fitness,DA:5D:5A:48:68:17,00001533-1412-efde-1523-785feabcd123
9,None,"iBeacon, Eddystone URL beacon - http://mrefer....",E2:7A:82:7A:9E:DF,"feaa,1786b941-062b-17b7-4df6-3f79361ad91f"
10,XY4+,"iBeacon, Dialog Semiconductor",E9:12:1D:F1:C3:0E,"fef5,e8d84b30-785f-0000-0000-0401f4ac4ea4"


# Dump Data

In [20]:
total_devices = pd.read_sql_query("SELECT * from devices", db)

device_names = pd.DataFrame(total_devices.device_name.value_counts()).sort_index()
device_names.columns = ["Count"]
device_names.to_csv('device_names.csv', index_label='Device Name')
device_names.index.names = ["Top Device Names"]

device_types = pd.DataFrame(total_devices.device_type.value_counts()).sort_index()
device_types.columns = ["Count"]
device_types.to_csv('device_types.csv', index_label='Device Type')
device_types.index.names = ["Top Device Types"]

print(f"Exported {device_names.count()[0]} device names to `device_names.csv`")
print(f"Exported {device_types.count()[0]} device names to `device_types.csv`")

Exported 537 device names to `device_names.csv`
Exported 122 device names to `device_types.csv`


## Count by Device Name

In [21]:
device_names.sort_values(by="Count", ascending=False).head()

,Count
Top Device Names,
Tile,104
Charge 2,32
Versa,23
H,16
Apple Pencil             ,15


## Count by Device Types

In [22]:
device_types.sort_values(by="Count", ascending=False).head()

,Count
Top Device Types,
"Apple, Nearby",6043
Microsoft,1716
iBeacon,1120
"Apple, Handoff",575
Google,348
